In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy.data import Field, BucketIterator, Iterator # type: ignore
from torchtext.legacy import data # type: ignore

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np
import pandas as pd

import random
import math
import time

In [2]:
!pip install torchtext

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import os
print(os.getcwd())


d:\Coding Assistant With Transformers\notebook


In [11]:
try:
    f = open(r'data\english_python_data.txt', 'r')
    file_lines = f.readlines()
    print(file_lines)
except FileNotFoundError:
    print("File not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


['# write a python program to add two numbers \n', 'num1 = 1.5\n', 'num2 = 6.3\n', 'sum = num1 + num2\n', "print(f'Sum: {sum}')\n", '\n', '\n', '# write a python function to add two user provided numbers and return the sum\n', 'def add_two_numbers(num1, num2):\n', '    sum = num1 + num2\n', '    return sum\n', '\n', '\n', '# write a program to find and print the largest among three numbers\n', '\n', 'num1 = 10\n', 'num2 = 12\n', 'num3 = 14\n', 'if (num1 >= num2) and (num1 >= num3):\n', '   largest = num1\n', 'elif (num2 >= num1) and (num2 >= num3):\n', '   largest = num2\n', 'else:\n', '   largest = num3\n', "print(f'largest:{largest}')\n", '\n', '\n', '# write a program to find and print the smallest among three numbers\n', 'num1 = 10\n', 'num2 = 12\n', 'num3 = 14\n', 'if (num1 <= num2) and (num1 <= num3):\n', '   smallest = num1\n', 'elif (num2 <= num1) and (num2 <= num3):\n', '   smallest = num2\n', 'else:\n', '   smallest = num3\n', "print(f'smallest:{smallest}')\n", '\n', '\n', '#

In [12]:
f = open(r'data\english_python_data.txt', 'r')
file_lines = f.readlines()

In [13]:
file_lines[:20]

['# write a python program to add two numbers \n',
 'num1 = 1.5\n',
 'num2 = 6.3\n',
 'sum = num1 + num2\n',
 "print(f'Sum: {sum}')\n",
 '\n',
 '\n',
 '# write a python function to add two user provided numbers and return the sum\n',
 'def add_two_numbers(num1, num2):\n',
 '    sum = num1 + num2\n',
 '    return sum\n',
 '\n',
 '\n',
 '# write a program to find and print the largest among three numbers\n',
 '\n',
 'num1 = 10\n',
 'num2 = 12\n',
 'num3 = 14\n',
 'if (num1 >= num2) and (num1 >= num3):\n',
 '   largest = num1\n']

In [14]:
dps = []
dp = None
for line in file_lines:
    if line[0] == "#":  # New question encountered
        if dp:  # Process the previous question and solution
            dp['Solution'] = ''.join(dp['Solution'])  # Join the solution lines
            dps.append(dp)  # Add the completed question-solution pair to the list
        dp = {"Question": None, "Solution": []}  # Initialize a new dictionary
        dp['Question'] = line[1:]  # Store the current question, without the '#'
    else:
        if dp:  # Ensure dp is initialized before appending solution lines
            dp['Solution'].append(line)  #

In [15]:
i = 0
for dp in dps:
  print('\n Question no: ', i+1)
  i+=1
  print(dp['Question'][1:])
  print(dp['Solution'])
  if i>49:
    break


 Question no:  1
write a python program to add two numbers 

num1 = 1.5
num2 = 6.3
sum = num1 + num2
print(f'Sum: {sum}')




 Question no:  2
write a python function to add two user provided numbers and return the sum

def add_two_numbers(num1, num2):
    sum = num1 + num2
    return sum




 Question no:  3
write a program to find and print the largest among three numbers


num1 = 10
num2 = 12
num3 = 14
if (num1 >= num2) and (num1 >= num3):
   largest = num1
elif (num2 >= num1) and (num2 >= num3):
   largest = num2
else:
   largest = num3
print(f'largest:{largest}')




 Question no:  4
write a program to find and print the smallest among three numbers

num1 = 10
num2 = 12
num3 = 14
if (num1 <= num2) and (num1 <= num3):
   smallest = num1
elif (num2 <= num1) and (num2 <= num3):
   smallest = num2
else:
   smallest = num3
print(f'smallest:{smallest}')




 Question no:  5
Write a python function to merge two given lists into one

def merge_lists(l1, l2):
    return l1 + l2




 Quest

In [16]:
print("Dataset size", len(dps))

Dataset size 4957


In [17]:
from tokenize import tokenize, untokenize
import io

def tokenize_python_code(python_code_str):
  python_tokens = list(tokenize(io.BytesIO(python_code_str.encode('utf-8')).readline))
  tokenized_output = []
  for i in range(0, len(python_tokens)):
    tokenized_output.append((python_tokens[i].type, python_tokens[i].string))
  return tokenized_output

tokenized_sample = tokenize_python_code(dps[1]['Solution'])
print(tokenized_sample)

[(67, 'utf-8'), (1, 'def'), (1, 'add_two_numbers'), (55, '('), (1, 'num1'), (55, ','), (1, 'num2'), (55, ')'), (55, ':'), (4, '\n'), (5, '    '), (1, 'sum'), (55, '='), (1, 'num1'), (55, '+'), (1, 'num2'), (4, '\n'), (1, 'return'), (1, 'sum'), (4, '\n'), (65, '\n'), (65, '\n'), (6, ''), (0, '')]


In [18]:
print(untokenize(tokenized_sample).decode('utf-8'))

def add_two_numbers (num1 ,num2 ):
    sum =num1 +num2 
    return sum 





In [19]:
import keyword

print(keyword.kwlist)

['False', 'None', 'True', 'and', 'as', 'assert', 'async', 'await', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except', 'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'nonlocal', 'not', 'or', 'pass', 'raise', 'return', 'try', 'while', 'with', 'yield']
